In [8]:
import collections

In [9]:
# Define sets of opcodes for different instruction types:
immediate_opcodes = {"ADDI", "ANDI", "ORI", "XORI", "SLTI", "SLL", "Scheduler.ipynb"}
load_opcodes      = {"LW"}
store_opcodes     = {"SW"}
branch_opcodes    = {"BEQ", "BNE"}
jump_opcodes      = {"J", "JR", "JAL"}

class Instruction:
    def __init__(self, line):
        # Remove commas and extra whitespace, then split into tokens.
        tokens = line.replace(",", "").split()
        if not tokens:
            return
        self.line = line.strip() if line != "NOP" else "SLL $0, $0, 0"
        self.latency = None if line != "NOP" else 0
        # Set default fields
        self.opcode = tokens[0].upper() if line != "NOP"  else "SLL"
        self.dest = None if line != "NOP" else "$0"
        self.rs = None if line != "NOP" else "$0"
        self.rt = None
        self.immediate = None if line != "NOP" else "0"
        # Flags 
        self.is_load = False
        self.is_store = False
        self.is_branch = False
        self.is_jump = False
        if line != "NOP":
            # Parse based on opcode type:
            if self.opcode in load_opcodes:
                # Format: LW $t, offset($s)
                # tokens[1] is the destination register (t)
                # tokens[2] is in the form offset($s)
                self.dest = tokens[1]
                operand = tokens[2]
                open_paren = operand.find('(')
                close_paren = operand.find(')')
                if open_paren != -1 and close_paren != -1:
                    self.immediate = operand[:open_paren]  # offset
                    self.rs = operand[open_paren+1:close_paren]  # base register
                else:
                    self.immediate = operand
                self.latency = 2
                self.is_load = True

            elif self.opcode in store_opcodes:
                # Format: SW $t, offset($s)
                # For a store, there is no destination; 
                # we treat $t as the register whose value is stored.
                self.rt = tokens[1]
                operand = tokens[2]
                self.latency = 0
                open_paren = operand.find('(')
                close_paren = operand.find(')')
                if open_paren != -1 and close_paren != -1:
                    self.immediate = operand[:open_paren]  # offset
                    self.rs = operand[open_paren+1:close_paren]  # base register
                else:
                    self.immediate = operand
                self.is_store = True

            elif self.opcode in branch_opcodes:
                # Format: BEQ $s, $t, label
                # Branches only have two registers.
                self.rs = tokens[1]
                self.rt = tokens[2]
                self.latency = 0
                if len(tokens) >= 4:
                    self.immediate = tokens[3]  # branch target
                self.is_branch = True

            elif self.opcode in jump_opcodes:
                self.latency = 0
                # Format: J label, JR $s, or JAL label
                if len(tokens) >= 2:
                    self.immediate = tokens[1]
                self.is_jump = True

            elif self.opcode in immediate_opcodes:
                # Immediate instructions: e.g. ADDI $t, $s, imm
                # Here, destination is the first operand (t), then rs, then immediate.
                self.latency = 1
                self.dest = tokens[1]
                self.rs = tokens[2]
                if len(tokens) >= 4:
                    self.immediate = tokens[3]

            else:
                # Assume R-type instruction: e.g. ADD $d, $s, $t
                # Parsed as: dest = tokens[1], rs = tokens[2], rt = tokens[3]
                self.dest = tokens[1]
                self.latency = 1
                self.rs = tokens[2]
                if len(tokens) >= 4:
                    self.rt = tokens[3]

    def __repr__(self):
        # Reconstruct a representation for debugging.
        parts = [self.opcode]
        if self.dest is not None:
            parts.append(self.dest)
        if self.rs is not None:
            parts.append(self.rs)
        if self.rt is not None:
            parts.append(self.rt)
        if self.immediate is not None:
            parts.append(self.immediate)
        return " ".join(parts)




In [16]:
def unpseudo(instructions):
    unpseudoed = []
    for instruction in instructions.splitlines():
        label = ''
        
        instruction = instruction.replace(",", "").strip()
        instruction = instruction.split()
        if ":" in instruction[0]:
            label = str(instruction[0]) + ' '
            first = str(instruction[1]).upper()
            instruction = instruction[1:]
        else:
            first = str(instruction[0]).upper()
            
        if first == 'SGT':
            # Transform SGT: sgt A, B, C  =>  slt A, C, B
            unpseudoed.append(label + 'SLT ' + str(instruction[1]) + ' ' + str(instruction[3]) + ' ' + str(instruction[2]))
        elif first == 'LI':
            # Transform LI: li A, imm  =>  ori A, $0, imm
            unpseudoed.append(label + 'ORI ' + str(instruction[1]) + ' $0 ' + str(instruction[2]))
        elif first == 'BLT':
            unpseudoed.append(label + 'ORI $30 ' + str(instruction[1]) + ' ' + str(instruction[2]))
            unpseudoed.append('BNE $30 $0 ' + str(instruction[3]))
        elif first == 'BLE':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[2]) + ' ' + str(instruction[1]))
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[3]))
        elif first == 'BGT':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[2]) + ' ' + str(instruction[1]))
            unpseudoed.append('BNE $30 $0 ' + str(instruction[3]))
        elif first == 'BGE':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[1]) + ' ' + str(instruction[2]))
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[3]))
        elif first == 'BLTZ':
            unpseudoed.append(label + 'SLTI $30 ' + str(instruction[1]) + ' 0')
            unpseudoed.append('BNE $30 $0 ' + str(instruction[2]))
        elif first == 'BGEZ':
            unpseudoed.append(label + 'SLTI $30 ' + str(instruction[1]) + ' 0')
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[2]))
        else:
            unpseudoed.append(label + ' '.join(instruction))
    return unpseudoed

def is_label(line):
    """Return True if the line is a label (ends with ':')."""
    return line.endswith(':')

def is_pseudo(line):
    """Returns if an instruction is pseudo or not"""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    pseudos = {"SGT", "LI", "BLT", "BLE", "BGT", "BGE", "BLTZ", "BGEZ"}
    return opcode in pseudos

def is_branch(line):
    """Return True if the line starts with a branch opcode."""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    branch_opcodes = {"BEQ", "BNE", "BLE", "BGT", "BLT", "BGE", "BLTZ", "BGEZ"}
    return opcode in branch_opcodes

def is_jump(line):
    """Return True if the line starts with a jump opcode."""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    jump_opcodes = {"J", "JR", "JAL"}
    return opcode in jump_opcodes

def is_boundary(line):
    """A line is a boundary if it is a label, branch, or jump."""
    return is_label(line) or is_branch(line) or is_jump(line)

def segment_code(code_str):
    """
    Accepts a string containing the assembly code.
    - Removes comments (ignores any text following '#').
    - Splits the code into lines.
    - Segments the code so that boundaries (labels, branches, jumps)
      are isolated as their own segments.
      
    Returns a list of segments (each segment is a list of lines).
    """
    segments = []
    current_segment = []
    
    # Split the input string into individual lines.
    lines = code_str.splitlines()
    for raw_line in lines:
        # Remove comments (anything after '#' is discarded) and trim whitespace.
        line = raw_line.split('#')[0].strip()
        if not line:
            continue  # Skip empty lines
        
        # If the instruction is pseudo, convert it.
        if is_pseudo(line):
            unpseudo_lines = []
            unpseudo_lines = unpseudo(line)
        else:
            unpseudo_lines = []
            unpseudo_lines = [line]
            
        for line1 in unpseudo_lines:
            if is_boundary(line1):
                # Finalize any existing non-boundary segment.
                if current_segment:
                    # If the boundary isn't a label, add it to the current segment.
                    if not is_label(line1):
                        current_segment.append(line1)
                        segments.append(current_segment)
                    else:
                        segments.append(current_segment)
                        segments.append([line1])
                    
                    current_segment = []
                else:
                    # If there's no current segment, the boundary becomes its own segment.
                    segments.append([line1])
            else:
                # Non-boundary line: add to the current segment.
                current_segment.append(line1)
    
    # Add any leftover non-boundary instructions as a final segment.
    if current_segment:
        segments.append(current_segment)
    
    return segments

# Example usage with your assembly code snippet:
assembly_code = """\
# Initialize registers
ORI $2 , $0, 0x0
ADDI $20, $0, 0xA
XORI $31, $0, 0x1
ANDI $5 , $0, 0x0
LW $10, 0x0($5)
LW $15, 0x0($5)
LOOP:
ADDI $2, $2, 1
SGT $25, $20, $2
BNE $25, $31, END
# Choose one of these Insertion based on your memory
# For Word addressable # For byte addressable
# ADD $5, $2, $0 # SLL $5, $2, 2
LW $16, 0x0($5)
SGT $26, $16, $10
BEQ $26, $0, MIN
OR $10, $16, $0
J LOOP
MIN:
SLT $27, $16, $15
BEQ $27, $0, LOOP
BEQ $27, $0, LOOP
ADD $15, $16, $0
J LOOP
END:
NOP

"""

segments = segment_code(assembly_code)

# Display the resulting segments:
for idx, segment in enumerate(segments):
    print(f"Segment {idx}:")
    for line in segment:
        print(f"  {line}")



Segment 0:
  ORI $2 , $0, 0x0
  ADDI $20, $0, 0xA
  XORI $31, $0, 0x1
  ANDI $5 , $0, 0x0
  LW $10, 0x0($5)
  LW $15, 0x0($5)
Segment 1:
  LOOP:
Segment 2:
  ADDI $2, $2, 1
  SLT $25 $2 $20
  BNE $25, $31, END
Segment 3:
  LW $16, 0x0($5)
  SLT $26 $10 $16
  BEQ $26, $0, MIN
Segment 4:
  OR $10, $16, $0
  J LOOP
Segment 5:
  MIN:
Segment 6:
  SLT $27, $16, $15
  BEQ $27, $0, LOOP
Segment 7:
  BEQ $27, $0, LOOP
Segment 8:
  ADD $15, $16, $0
  J LOOP
Segment 9:
  END:
Segment 10:
  NOP


In [ ]:
import collections

#------------------------------------------------------------------------------
# Helper Functions and Definitions
#------------------------------------------------------------------------------

NOP = Instruction("NOP")


def is_lw(instr):
    """Return True if the instruction is a load-word instruction."""
    return instr.opcode == "LW"

def build_dependency_graph(instructions):
    """
    Build a dependency graph as a list of sets.
    dependencies[i] is the set of instruction indices that must come before instruction i.
    """
    n = len(instructions)
    dependencies = [set() for _ in range(n)]
    for i in range(n):
        for j in range(i):
            # If instr i reads a register that instr j writes, add dependency.
            if instructions[j].dest:
                if instructions[i].rs and instructions[i].rs == instructions[j].dest:
                    dependencies[i].add(j)
                if instructions[i].rt and instructions[i].rt == instructions[j].dest:
                    dependencies[i].add(j)
    return dependencies



#------------------------------------------------------------------------------
# Main Rescheduling Routine
#------------------------------------------------------------------------------
def reschedule(instructions):
    unscheduled = set(range(len(instructions)))
    scheduled = {}
    packets = []
    cycle = 0
    while unscheduled:
        ready = []
        for i in unscheduled:
            # Check if all dependencies are scheduled.
            if not all(dep in scheduled for dep in graph[i]):
                continue
            # If an LW was scheduled only one cycle before it. Delay it
            delay_due_to_lw = False
            for dep in graph[i]:
                if is_lw(instructions[dep]) and scheduled[dep] >= cycle -1:
                    delay_due_to_lw = True
                    break
            if delay_due_to_lw:
                continue
            #Append the index of the ready instruction
            ready.append(i)
        # If no instruction is ready, add an empty packet
        if not ready:
            packets.append([NOP, NOP])
            cycle += 1
            continue
        candidate1 = None
        candidate2 = None
        for i in ready:
            if not is_branch(instructions[i].line) and not is_jump(instructions[i].line):
                candidate1 = i
                break
        for i in ready:
            if candidate1 is not None and i == candidate1:
                continue
            candidate2 = i
            break
        slot1 = instructions[candidate1] if candidate1 is not None else NOP
        slot2 = instructions[candidate2] if candidate2 is not None else NOP
        
        #Append packet and update scheduled 
        packets.append([slot1, slot2])
        if candidate1 is not None:
            scheduled[candidate1] = cycle
            unscheduled.remove(candidate1)
        if candidate2 is not None and candidate2 != candidate1:
            scheduled[candidate2] = cycle
            unscheduled.remove(candidate2)
        cycle += 1
    final_schedule = [instr for packet in  packets for instr in packet]
    return final_schedule       
        
final_code = []  # This will collect the final scheduled code as a list of Instruction objects.

for segment in segments:
    # Convert each line to an Instruction object.
    if len(segment) >1:
        segment_instrs = [Instruction(line) for line in segment]
        graph = build_dependency_graph(segment_instrs)
        final_schedule = reschedule(segment_instrs)
        for i in final_schedule:
            final_code.append(i)
    else:
        if is_branch(segment[0]):
            final_code.append(NOP.line)
            final_code.append(segment[0])
        elif is_jump(segment[0]):
            final_code.append(NOP.line)
            final_code.append(segment[0])
        elif is_label(segment[0]):
            final_code.append(segment[0])
        else:
            final_code.append(segment[0])
            final_code.append(NOP.line)




Final Scheduled Code:
00: [ORI $2 $0 0x0, ADDI $20 $0 0xA, XORI $31 $0 0x1, ANDI $5 $0 0x0, LW $10 $5 0x0, LW $15 $5 0x0]
01: LOOP:
02: [ADDI $2 $2 1, SLL $0 $0 0, SLT $25 $2 $20, SLL $0 $0 0, SLL $0 $0 0, BNE $25 $31 END]
03: [LW $16 $5 0x0, SLL $0 $0 0, SLL $0 $0 0, SLL $0 $0 0, SLT $26 $10 $16, SLL $0 $0 0, SLL $0 $0 0, BEQ $26 $0 MIN]
04: [OR $10 $16 $0, J LOOP]
05: MIN:
06: [SLT $27 $16 $15, SLL $0 $0 0, SLL $0 $0 0, BEQ $27 $0 LOOP]
07: SLL $0, $0, 0
08: BEQ $27, $0, LOOP
09: [ADD $15 $16 $0, J LOOP]
10: END:
11: NOP
12: SLL $0, $0, 0
